In [33]:
class Variable:
    """A mathematical variableession with an operator and 0 or more arguments.
    op is a str like '+' or 'sin'; args are Variableessions.
    Variable('x') or Symbol('x') creates a symbol (a nullary Variable).
    Variable('-', x) creates a unary; Variable('+', x, 1) creates a binary."""

    def __init__(self, op, *args):
        self.op = str(op)
        self.args = args

    # Operator overloads
    def __neg__(self):
        return Variable('-', self)

    def __pos__(self):
        return Variable('+', self)

    def __invert__(self):
        return Variable('~', self)

    def __add__(self, rhs):
        return Variable('+', self, rhs)

    def __sub__(self, rhs):
        return Variable('-', self, rhs)

    def __mul__(self, rhs):
        return Variable('*', self, rhs)

    def __pow__(self, rhs):
        return Variable('**', self, rhs)

    def __mod__(self, rhs):
        return Variable('%', self, rhs)

    def __and__(self, rhs):
        return Variable('&', self, rhs)

    def __xor__(self, rhs):
        return Variable('^', self, rhs)

    def __rshift__(self, rhs):
        return Variable('>>', self, rhs)

    def __lshift__(self, rhs):
        return Variable('<<', self, rhs)

    def __truediv__(self, rhs):
        return Variable('/', self, rhs)

    def __floordiv__(self, rhs):
        return Variable('//', self, rhs)

    def __matmul__(self, rhs):
        return Variable('@', self, rhs)

    def __or__(self, rhs):
        """Allow both P | Q, and P |'==>'| Q."""
        if isinstance(rhs, Variableession):
            return Variable('|', self, rhs)
        else:
            return PartialVariable(rhs, self)

    # Reverse operator overloads
    def __radd__(self, lhs):
        return Variable('+', lhs, self)

    def __rsub__(self, lhs):
        return Variable('-', lhs, self)

    def __rmul__(self, lhs):
        return Variable('*', lhs, self)

    def __rdiv__(self, lhs):
        return Variable('/', lhs, self)

    def __rpow__(self, lhs):
        return Variable('**', lhs, self)

    def __rmod__(self, lhs):
        return Variable('%', lhs, self)

    def __rand__(self, lhs):
        return Variable('&', lhs, self)

    def __rxor__(self, lhs):
        return Variable('^', lhs, self)

    def __ror__(self, lhs):
        return Variable('|', lhs, self)

    def __rrshift__(self, lhs):
        return Variable('>>', lhs, self)

    def __rlshift__(self, lhs):
        return Variable('<<', lhs, self)

    def __rtruediv__(self, lhs):
        return Variable('/', lhs, self)

    def __rfloordiv__(self, lhs):
        return Variable('//', lhs, self)

    def __rmatmul__(self, lhs):
        return Variable('@', lhs, self)

    def __call__(self, *args):
        """Call: if 'f' is a Symbol, then f(0) == Variable('f', 0)."""
        if self.args:
            raise ValueError('Can only do a call for a Symbol, not an Variable')
        else:
            return Variable(self.op, *args)

    # Equality and repr
    def __eq__(self, other):
        """x == y' evaluates to True or False; does not build an Variable."""
        return isinstance(other, Variable) and self.op == other.op and self.args == other.args

    def __lt__(self, other):
        return isinstance(other, Variable) and str(self) < str(other)

    def __hash__(self):
        return hash(self.op) ^ hash(self.args)

    def __repr__(self):
        op = self.op
        args = [str(arg) for arg in self.args]
        if op.isidentifier():  # f(x) or f(x, y)
            return '{}({})'.format(op, ', '.join(args)) if args else op
        elif len(args) == 1:  # -x or -(x + 1)
            return op + args[0]
        else:  # (x - y)
            opp = (' ' + op + ' ')
            return '(' + opp.join(args) + ')'

In [47]:
def is_var(x):
    """A var is an Variable with no args and a lowercase symbol as the op."""
    return isinstance(x, Variable) and not x.args and x.op[0].islower()

In [114]:
def unification_var(var, val, subst):
    if var in subst :   
        return unification(subst[var], val, subst)
    elif isinstance(val, str) and val in subst : 
        return unification(var, subst[val], subst)
    else :
        subst[var] = val ; return subst

def unification(first, second, subst):

    if subst is False : return False
    elif isinstance(first, str) and isinstance(second, str) and first == second : return subst
    elif isinstance(first, str) and is_var(second) : return unification_var(second, first, subst)
    elif isinstance(second, str) and is_var(first) : return unification_var(first, second, subst)
    elif isinstance(first, list) and isinstance(second, list) : 
        if len(first) == 0 and len(second) == 0 : return subst
        return unification(first[1:],second[1:], unification(first[0], second[0], subst))
    else: return False

In [124]:
unification(['p', 'q', 'a+b'], ['p', 'q',Variable('x')],{}) 

{x: 'a+b'}

In [125]:
unification(['p', 'r', 'a+b'], ['p', 'q',Variable('x')],{}) 

False

In [127]:
unification(['p', 'q', 'a'], ['p', Variable('y'),Variable('x')],{}) 

{x: 'a', y: 'q'}